# WNBA 2026 Draft Insights 

With the NCAA women’s basketball season entering its final stretch ahead of tournament play, this notebook explores WNBA Draft–related metrics for current prospects.

This Jupyter notebook uses SQL to analyze player statistics and highlight analytical insights across a subset of draft-eligible players. The underlying data was collected via web scraping from publicly available ESPN sources.

This analysis is intended for educational and portfolio purposes only and is not meant to inform or guide decision-making by any WNBA team or organization.

# Acquiring the Data 

In [51]:
import requests

url = "https://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/teams"
data = requests.get(url).json()
print(data.keys())



dict_keys(['sports'])


In [52]:
import json
print(json.dumps(data, indent=2)[:3000])


{
  "sports": [
    {
      "id": "40",
      "uid": "s:40",
      "name": "Basketball",
      "slug": "basketball",
      "leagues": [
        {
          "id": "54",
          "uid": "s:40~l:54",
          "name": "NCAA Women's Basketball",
          "abbreviation": "NCAAW",
          "shortName": "NCAAW",
          "slug": "womens-college-basketball",
          "teams": [
            {
              "team": {
                "id": "44",
                "uid": "s:40~l:54~t:44",
                "slug": "american-university-eagles",
                "abbreviation": "AMER",
                "displayName": "American University Eagles",
                "shortDisplayName": "American",
                "name": "Eagles",
                "nickname": "American",
                "location": "American University",
                "color": "c41130",
                "alternateColor": "c8102e",
                "isActive": true,
                "isAllStar": false,
                "logos": [
           

In [53]:
import pandas as pd
teams = []

for sport in data["sports"]:
    for league in sport["leagues"]:
        for t in league["teams"]:
            team = t["team"]
            teams.append({
                "team_id": team["id"],
                "team_name": team["displayName"]
            })

teams_df = pd.DataFrame(teams)
teams_df.head()


,team_id,team_name
0,44,American University Eagles
1,9,Arizona State Sun Devils
2,12,Arizona Wildcats
3,8,Arkansas Razorbacks
4,2,Auburn Tigers


In [54]:
team_id = teams_df.iloc[0]["team_id"]
roster_url = f"https://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/teams/{team_id}/roster"

roster_data = requests.get(roster_url).json()

print(roster_data.keys())


dict_keys(['timestamp', 'status', 'season', 'athletes', 'coach', 'team'])


In [55]:
players = []

for athlete in roster_data["athletes"]:
    players.append({
        "athlete_id": athlete["id"],
        "name": athlete["displayName"],
        "position": athlete.get("position", {}).get("abbreviation"),
        "height": athlete.get("height"),
        "class": athlete.get("class"),
        "team": teams_df.iloc[0]["team_name"]
    })

players_df = pd.DataFrame(players)
players_df.head()


,athlete_id,name,position,height,class,team
0,5242706,Elizabeth Archer,G,72.0,None,American University Eagles
1,5242708,Mary Bolesky,G,64.0,None,American University Eagles
2,5242705,Molly Driscoll,G,68.0,None,American University Eagles
3,5314863,Kayla Greyvensteyn,G,69.0,None,American University Eagles
4,5314862,Andrea Jude,G,69.0,None,American University Eagles


In [56]:
athlete_id = players_df.iloc[0]["athlete_id"]

stats_url = f"https://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/athletes/{athlete_id}/statistics"

stats_data = requests.get(stats_url).json()

import json
print(json.dumps(stats_data, indent=2)[:3000])


{
  "code": 404,
  "message": "Error invoking GET: http://sports.core.api.espn.pvt/v2/sports/basketball/leagues/womens-college-basketball/athletes/5242706/season, status=404, duration=6 ms, message=null, cache=CACHE_MISS"
}


In [57]:
athlete_id = players_df.iloc[0]["athlete_id"]

stats_url = f"https://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/athletes/{athlete_id}/statistics"
params = {"season": 2024}  # try 2025 if you're looking at current season
stats_data = requests.get(stats_url, params=params).json()

print(stats_data.keys())
print(stats_data.get("code"), stats_data.get("message"))


dict_keys(['code', 'message'])
404 Error invoking GET: http://sports.core.api.espn.pvt/v2/sports/basketball/leagues/womens-college-basketball/athletes/5242706/season, status=404, duration=3 ms, message=null, cache=CACHE_MISS


In [58]:
url = "https://www.espn.com/womens-college-basketball/stats/player/_/season/2026"
tables = pd.read_html(url)

len(tables), tables[0].head()


(2,
    RK               Name
 0   1     Audi CrooksISU
 1   2  Mikayla BlakesVAN
 2   3   Hannah HidalgoND
 3   4    Mia NicastroWIU
 4   5      Shay CiezkiIU)

In [59]:
all_pages = []
page = 1

while True:
    url = f"https://www.espn.com/womens-college-basketball/stats/player/_/season/2026/page/{page}"
    try:
        df = pd.read_html(url)[0]
        if df.empty:
            break
        df["page"] = page
        all_pages.append(df)
        page += 1
        time.sleep(0.5)  
    except ValueError:
        
        break

full_df = pd.concat(all_pages, ignore_index=True)
full_df.to_csv("espn_wcb_player_stats_season_2026.csv", index=False)

full_df.head()


,RK,Name,page
0,1,Audi CrooksISU,1
1,2,Mikayla BlakesVAN,1
2,3,Hannah HidalgoND,1
3,4,Mia NicastroWIU,1
4,5,Shay CiezkiIU,1


In [60]:
import time 
all_pages = []
page = 1

while True:
    url = f"https://www.espn.com/womens-college-basketball/stats/player/_/season/2026/page/{page}"
    try:
        tables = pd.read_html(url)

        if len(tables) < 2:
            break

        left = tables[0]
        right = tables[1]

        df = pd.concat([left, right], axis=1)

        if df.empty:
            break

        df["page"] = page
        all_pages.append(df)

        page += 1
        time.sleep(0.5)

    except ValueError:
        break

full_df = pd.concat(all_pages, ignore_index=True)
full_df.to_csv("espn_wcb_player_stats_season_2026.csv", index=False)

full_df.head()


,RK,Name,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,3P%,FTM,FTA,FT%,REB,AST,STL,BLK,TO,page
0,1,Audi CrooksISU,C,21,28.6,26.6,10.9,16.1,67.6,0.0,...,14.3,4.8,6.8,70.4,7.5,1.7,0.4,0.9,2.3,1
1,2,Mikayla BlakesVAN,G,22,33.3,24.9,8.6,19.1,44.9,2.3,...,31.4,5.4,6.5,83.2,3.9,4.5,3.0,0.2,2.8,1
2,3,Hannah HidalgoND,G,21,35.6,24.8,9.2,19.5,47.4,1.1,...,22.8,5.2,6.4,81.3,5.8,5.5,5.7,0.3,3.2,1
3,4,Mia NicastroWIU,F,20,34.1,23.6,8.7,17.0,51.2,0.8,...,37.5,5.5,6.0,90.8,10.2,1.1,1.4,0.7,1.8,1
4,5,Shay CiezkiIU,G,22,35.8,23.1,8.4,15.7,53.6,2.6,...,46.0,3.7,4.1,91.1,3.1,2.6,1.5,0.2,3.8,1


In [61]:
import re


full_df["Name"] = full_df["Name"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()


full_df["team_abbr"] = full_df["Name"].str.extract(r"([A-Z]{2,5})$")[0]


full_df["player_name"] = full_df["Name"].str.replace(r"([A-Z]{2,5})$", "", regex=True).str.strip()

full_df[["player_name", "team_abbr"]].head()


,player_name,team_abbr
0,Audi Crooks,ISU
1,Mikayla Blakes,VAN
2,Hannah Hidalgo,ND
3,Mia Nicastro,WIU
4,Shay Ciezki,IU


In [62]:
full_df.head()

,RK,Name,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FTA,FT%,REB,AST,STL,BLK,TO,page,team_abbr,player_name
0,1,Audi CrooksISU,C,21,28.6,26.6,10.9,16.1,67.6,0.0,...,6.8,70.4,7.5,1.7,0.4,0.9,2.3,1,ISU,Audi Crooks
1,2,Mikayla BlakesVAN,G,22,33.3,24.9,8.6,19.1,44.9,2.3,...,6.5,83.2,3.9,4.5,3.0,0.2,2.8,1,VAN,Mikayla Blakes
2,3,Hannah HidalgoND,G,21,35.6,24.8,9.2,19.5,47.4,1.1,...,6.4,81.3,5.8,5.5,5.7,0.3,3.2,1,ND,Hannah Hidalgo
3,4,Mia NicastroWIU,F,20,34.1,23.6,8.7,17.0,51.2,0.8,...,6.0,90.8,10.2,1.1,1.4,0.7,1.8,1,WIU,Mia Nicastro
4,5,Shay CiezkiIU,G,22,35.8,23.1,8.4,15.7,53.6,2.6,...,4.1,91.1,3.1,2.6,1.5,0.2,3.8,1,IU,Shay Ciezki


In [63]:
full_df.columns

Index(['RK', 'Name', 'POS', 'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM',
       '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'REB', 'AST', 'STL', 'BLK', 'TO',
       'page', 'team_abbr', 'player_name'],
      dtype='object')

This data set does not have advanced stats, but we're going to add two advacned stats that can help us see how efficient players are.

In [64]:
full_df['Effective_FG%'] = ((full_df['FGM'] + 0.5 * full_df['3PM']) / full_df['FGA']) * 100


In [65]:
full_df['TS%'] = (full_df['PTS'] / (2 * (full_df['FGA'] + 0.44 * full_df['FTA']))) * 100
 

In [66]:
full_df

,RK,Name,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,REB,AST,STL,BLK,TO,page,team_abbr,player_name,Effective_FG%,TS%
0,1,Audi CrooksISU,C,21,28.6,26.6,10.9,16.1,67.6,0.0,...,7.5,1.7,0.4,0.9,2.3,1,ISU,Audi Crooks,67.701863,69.662686
1,2,Mikayla BlakesVAN,G,22,33.3,24.9,8.6,19.1,44.9,2.3,...,3.9,4.5,3.0,0.2,2.8,1,VAN,Mikayla Blakes,51.047120,56.693989
2,3,Hannah HidalgoND,G,21,35.6,24.8,9.2,19.5,47.4,1.1,...,5.8,5.5,5.7,0.3,3.2,1,ND,Hannah Hidalgo,50.000000,55.565514
3,4,Mia NicastroWIU,F,20,34.1,23.6,8.7,17.0,51.2,0.8,...,10.2,1.1,1.4,0.7,1.8,1,WIU,Mia Nicastro,53.529412,60.081466
4,5,Shay CiezkiIU,G,22,35.8,23.1,8.4,15.7,53.6,2.6,...,3.1,2.6,1.5,0.2,3.8,1,IU,Shay Ciezki,61.783439,65.984918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3223,3223,Jah'Nae StevensCCSU,F,17,5.7,0.4,0.2,0.6,27.3,0.0,...,1.2,0.1,0.1,0.1,0.4,65,CCSU,Jah'Nae Stevens,33.333333,29.069767
3224,3225,Olivia FernandezBRY,G,20,6.6,0.4,0.1,0.6,9.1,0.1,...,1.2,0.7,0.6,0.0,0.6,65,BRY,Olivia Fernandez,25.000000,27.322404
3225,3226,Saige RandolphCAN,G,18,1.8,0.2,0.1,0.2,25.0,0.0,...,0.2,0.1,0.1,0.0,0.2,65,CAN,Saige Randolph,50.000000,30.120482
3226,3227,Dieynaba SyllaUCR,F,16,7.0,0.1,0.1,0.3,25.0,0.0,...,0.7,0.0,0.1,0.2,0.5,65,UCR,Dieynaba Sylla,33.333333,16.666667


Let's go! We have our dataframe!

In [67]:
final_df = full_df.copy()
final_df.to_csv("wcb_stats_2026_scoring.csv", index=False)

In [68]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [69]:
import sqlite3


conn = sqlite3.connect("wnba_draft.db")
df = pd.read_csv("wcb_stats_2026_scoring.csv")

df.to_sql("prospects_raw", conn, if_exists="replace", index=False)


3228

In [70]:
%load_ext sql
%sql sqlite:///wnba_draft.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [71]:
%%sql
-- ============================================================
-- 0) Build clean table with schema fixes (TO -> TOV)
-- ============================================================

DROP TABLE IF EXISTS prospects_clean;

CREATE TABLE prospects_clean AS
SELECT
  RK,
  player_name,
  team_abbr,
  POS,
  GP,
  MIN,
  PTS,
  FGM,
  FGA,
  [FG%],
  [3PM],
  [3PA],
  [3P%],
  FTM,
  FTA,
  [FT%],
  REB,
  AST,
  STL,
  BLK,
  "TO" AS TOV
FROM prospects_raw;


 * sqlite:///wnba_draft.db
Done.
Done.


[]

# Features Engineering

Let's try to create more draft metrics as best as we can!

In [72]:
%%sql
SELECT
  RK,
  player_name AS Name,
  team_abbr AS Team,
  POS,
  GP, MIN,
  PTS,
  FGM, FGA, [FG%],
  [3PM] AS TPM, [3PA] AS TPA, [3P%],
  FTM, FTA, [FT%],
  REB, AST, STL, BLK, TOV,

  (PTS * 40.0) / NULLIF(MIN, 0) AS pts_per40,
  (REB * 40.0) / NULLIF(MIN, 0) AS reb_per40,
  (AST * 40.0) / NULLIF(MIN, 0) AS ast_per40,
  (STL * 40.0) / NULLIF(MIN, 0) AS stl_per40,
  (BLK * 40.0) / NULLIF(MIN, 0) AS blk_per40,
  (TOV * 40.0) / NULLIF(MIN, 0) AS tov_per40,

  ([3PM] * 1.0) / NULLIF(GP, 0) AS threes_per_game,

  ((FGM + 0.5 * [3PM]) * 1.0) / NULLIF(FGA, 0) AS efg,
  (PTS * 1.0) / NULLIF(2.0 * (FGA + 0.44 * FTA), 0) AS ts,

  ([3PA] * 1.0) / NULLIF(FGA, 0) AS three_rate,
  (FTA * 1.0) / NULLIF(FGA, 0) AS ft_rate,

  (AST * 1.0) / NULLIF(TOV, 0) AS ast_to,
  ((STL + BLK) * 40.0) / NULLIF(MIN, 0) AS stocks_per40

FROM prospects
WHERE GP IS NOT NULL
LIMIT 5;

 * sqlite:///wnba_draft.db
Done.


RK,Name,Team,POS,GP,MIN,PTS,FGM,FGA,FG%,TPM,TPA,3P%,FTM,FTA,FT%,REB,AST,STL,BLK,TOV,pts_per40,reb_per40,ast_per40,stl_per40,blk_per40,tov_per40,threes_per_game,efg,ts,three_rate,ft_rate,ast_to,stocks_per40
1,Audi Crooks,ISU,C,21,28.6,26.6,10.9,16.1,67.6,0.0,0.3,14.3,4.8,6.8,70.4,7.5,1.7,0.4,0.9,2.3,37.2027972027972,10.489510489510488,2.3776223776223775,0.5594405594405594,1.2587412587412588,3.2167832167832167,0.0,0.6770186335403726,0.6966268594175571,0.018633540372670804,0.42236024844720493,0.7391304347826088,1.8181818181818181
2,Hannah Hidalgo,ND,G,21,35.6,24.8,9.2,19.5,47.4,1.1,4.8,22.8,5.2,6.4,81.3,5.8,5.5,5.7,0.3,3.2,27.865168539325843,6.51685393258427,6.179775280898876,6.404494382022472,0.33707865168539325,3.595505617977528,0.05238095238095238,0.5,0.5556551353289121,0.24615384615384614,0.3282051282051282,1.71875,6.741573033707865
3,Mikayla Blakes,VAN,G,21,33.0,24.7,8.5,18.9,45.1,2.3,7.4,31.6,5.3,6.3,83.5,3.8,4.5,3.1,0.2,2.6,29.939393939393938,4.606060606060606,5.454545454545454,3.757575757575758,0.24242424242424243,3.1515151515151514,0.10952380952380951,0.5105820105820107,0.5698597268364711,0.39153439153439157,0.33333333333333337,1.7307692307692306,4.0
4,Mia Nicastro,WIU,F,20,34.1,23.6,8.7,17.0,51.2,0.8,2.0,37.5,5.5,6.0,90.8,10.2,1.1,1.4,0.7,1.8,27.683284457478006,11.964809384164223,1.2903225806451613,1.6422287390029324,0.8211143695014662,2.1114369501466275,0.04,0.5352941176470588,0.6008146639511202,0.11764705882352941,0.35294117647058826,0.6111111111111112,2.4633431085043984
5,Shay Ciezki,IU,G,22,35.8,23.1,8.4,15.7,53.6,2.6,5.6,46.0,3.7,4.1,91.1,3.1,2.6,1.5,0.2,3.8,25.81005586592179,3.463687150837989,2.905027932960894,1.6759776536312851,0.223463687150838,4.245810055865922,0.11818181818181818,0.6178343949044587,0.6598491773308959,0.35668789808917195,0.2611464968152866,0.6842105263157895,1.8994413407821231


Perfect, this shows that our base query runs and we can make tables from there.

In [73]:
%%sql
DROP TABLE IF EXISTS prospect_features_base;

CREATE TABLE prospect_features_base AS
SELECT
  RK,
  player_name AS Name,
  team_abbr   AS Team,
  POS,
  GP, MIN,
  PTS,
  FGM, FGA, [FG%],
  [3PM] AS TPM, [3PA] AS TPA, [3P%],
  FTM, FTA, [FT%],
  REB, AST, STL, BLK, TOV,

  (PTS * 40.0) / NULLIF(MIN, 0)           AS pts_per40,
  (REB * 40.0) / NULLIF(MIN, 0)           AS reb_per40,
  (AST * 40.0) / NULLIF(MIN, 0)           AS ast_per40,
  (STL * 40.0) / NULLIF(MIN, 0)           AS stl_per40,
  (BLK * 40.0) / NULLIF(MIN, 0)           AS blk_per40,
  (TOV * 40.0) / NULLIF(MIN, 0)           AS tov_per40,

  ([3PM] * 1.0) / NULLIF(GP, 0)           AS threes_per_game,

  ((FGM + 0.5 * [3PM]) * 1.0) / NULLIF(FGA, 0)                 AS efg,
  (PTS * 1.0) / NULLIF(2.0 * (FGA + 0.44 * FTA), 0)            AS ts,

  ([3PA] * 1.0) / NULLIF(FGA, 0)          AS three_rate,
  (FTA * 1.0) / NULLIF(FGA, 0)            AS ft_rate,

  (AST * 1.0) / NULLIF(TOV, 0)            AS ast_to,
  ((STL + BLK) * 40.0) / NULLIF(MIN, 0)   AS stocks_per40

FROM prospects
WHERE GP IS NOT NULL;

 * sqlite:///wnba_draft.db
Done.
Done.


[]

We're going to create z scores because this metric turns each stat into “how far above or below average this player is compared to the rest of the draft class.” That lets us fairly combine totally different stats (points, steals, efficiency, turnovers) into one draft score.

In [74]:
%%sql
DROP TABLE IF EXISTS prospect_features_z;

CREATE TABLE prospect_features_z AS
WITH agg AS (
  SELECT
    AVG(pts_per40)       AS m_pts,
    SQRT(MAX(AVG(pts_per40*pts_per40) - AVG(pts_per40)*AVG(pts_per40), 0)) AS s_pts,

    AVG(ts)              AS m_ts,
    SQRT(MAX(AVG(ts*ts) - AVG(ts)*AVG(ts), 0)) AS s_ts,

    AVG(stocks_per40)    AS m_stocks,
    SQRT(MAX(AVG(stocks_per40*stocks_per40) - AVG(stocks_per40)*AVG(stocks_per40), 0)) AS s_stocks,

    AVG(ast_per40)       AS m_ast,
    SQRT(MAX(AVG(ast_per40*ast_per40) - AVG(ast_per40)*AVG(ast_per40), 0)) AS s_ast,

    AVG(reb_per40)       AS m_reb,
    SQRT(MAX(AVG(reb_per40*reb_per40) - AVG(reb_per40)*AVG(reb_per40), 0)) AS s_reb,

    AVG(tov_per40)       AS m_tov,
    SQRT(MAX(AVG(tov_per40*tov_per40) - AVG(tov_per40)*AVG(tov_per40), 0)) AS s_tov,

    AVG(three_rate)      AS m_3r,
    SQRT(MAX(AVG(three_rate*three_rate) - AVG(three_rate)*AVG(three_rate), 0)) AS s_3r,

    AVG(ft_rate)         AS m_ftr,
    SQRT(MAX(AVG(ft_rate*ft_rate) - AVG(ft_rate)*AVG(ft_rate), 0)) AS s_ftr
  FROM prospect_features_base
)
SELECT
  b.*,

  -- z-scores (higher is better, except turnovers)
  (b.pts_per40    - a.m_pts)   / NULLIF(a.s_pts, 0)   AS z_scoring,
  (b.ts           - a.m_ts)    / NULLIF(a.s_ts, 0)    AS z_efficiency,
  (b.stocks_per40 - a.m_stocks)/ NULLIF(a.s_stocks, 0) AS z_defense,
  (b.ast_per40    - a.m_ast)   / NULLIF(a.s_ast, 0)   AS z_playmaking,
  (b.reb_per40    - a.m_reb)   / NULLIF(a.s_reb, 0)   AS z_rebounding,
  (b.three_rate   - a.m_3r)    / NULLIF(a.s_3r, 0)    AS z_spacing,
  (b.ft_rate      - a.m_ftr)   / NULLIF(a.s_ftr, 0)   AS z_rim_pressure,

  -- turnovers: lower is better -> flip sign
  -1.0 * ((b.tov_per40 - a.m_tov) / NULLIF(a.s_tov, 0)) AS z_ball_security

FROM prospect_features_base b
CROSS JOIN agg a;

 * sqlite:///wnba_draft.db
Done.
Done.


[]

In [75]:
%%sql
SELECT Name, POS, z_scoring, z_efficiency, z_defense, z_ball_security
FROM prospect_features_z
LIMIT 10;

 * sqlite:///wnba_draft.db
Done.


Name,POS,z_scoring,z_efficiency,z_defense,z_ball_security
Audi Crooks,C,5.21945068115305,2.4689345290766553,-0.5076200976922243,-0.06435765264936695
Hannah Hidalgo,G,3.2271824226068433,0.7807211995797413,4.005349029183402,-0.39352959715161795
Mikayla Blakes,G,3.6697373339829937,0.9508289348589143,1.492318087993738,-0.007628988354788144
Mia Nicastro,F,3.1883757944938815,1.3215312000722441,0.08375947334394705,0.8963697265330159
Shay Ciezki,G,2.7887053858217006,2.028501715134058,-0.433134504184367,-0.9587509193208792
Maggie Doogan,F,2.8170397497451374,1.9219537984516464,0.23521408556256176,-1.2417668687225636
Hannah Wickstrom,G,3.1983560711735954,1.1760240214889672,1.4248355111761133,-0.2545304317262852
Liv McGill,G,2.5517482505776896,0.2221083803654502,1.0314944441152782,-2.207957180501894
Naomi White,G,2.3484699827807582,0.627359193879492,0.24433555259867182,-0.5708101267605995
Jaloni Cambridge,G,3.41337997694088,1.1138613836482425,0.7922318514726004,-0.6438397782104945


In [76]:
%%sql
DROP TABLE IF EXISTS prospect_features;

CREATE TABLE prospect_features AS
SELECT
  *,
  (
    0.22 * z_scoring +
    0.20 * z_efficiency +
    0.18 * z_defense +
    0.15 * z_playmaking +
    0.10 * z_rebounding +
    0.08 * z_spacing +
    0.07 * z_rim_pressure +
    0.10 * z_ball_security
  ) AS draft_score
FROM prospect_features_z;

 * sqlite:///wnba_draft.db
Done.
Done.


[]

In [77]:
%%sql
SELECT RK, Name, Team, POS, draft_score, pts_per40, ts, stocks_per40, ast_per40, tov_per40
FROM prospect_features
ORDER BY draft_score DESC
LIMIT 25;

 * sqlite:///wnba_draft.db
Done.


RK,Name,Team,POS,draft_score,pts_per40,ts,stocks_per40,ast_per40,tov_per40
30,Sarah Strong,CONN,F,2.5767056066527725,27.985347985347985,0.68863570810499,7.472527472527473,6.593406593406593,2.93040293040293
243,MiLaysia Fulwiley,LSU,G,2.22962359013319,26.857142857142858,0.5828373015873015,8.380952380952381,6.857142857142857,4.190476190476191
2,Hannah Hidalgo,ND,G,1.9046186301234096,27.865168539325843,0.5556551353289121,6.741573033707865,6.179775280898876,3.595505617977528
36,Avery Koenen,NDSU,F,1.8563324406905992,26.57243816254417,0.6839348079161817,3.8162544169611308,3.1095406360424027,2.1201413427561837
13,Brooklyn Meyer,SDST,F,1.7061891501000708,29.61937716262976,0.6694194194194194,3.8754325259515574,3.8754325259515574,2.768166089965398
1,Audi Crooks,ISU,C,1.593417378619174,37.2027972027972,0.6966268594175571,1.8181818181818181,2.3776223776223775,3.2167832167832167
57,Taylee Chirrick,MTST,G,1.592722945225973,24.16382252559727,0.5661463664278402,5.733788395904437,4.914675767918089,3.6860068259385663
55,Timaya Lewis-Eutsey,MRSH,G,1.5426825851604788,25.942028985507246,0.5603556223390934,5.942028985507246,5.652173913043478,4.63768115942029
3,Mikayla Blakes,VAN,G,1.5354694212817774,29.939393939393938,0.5698597268364711,4.0,5.454545454545454,3.1515151515151514
230,Antoniette Emma-Nnopu,WEB,F,1.522098250111292,21.59695817490494,0.6225885654156436,4.866920152091255,4.410646387832699,4.258555133079848


In [78]:
%%sql
SELECT RK, Name, Team, POS, draft_score, pts_per40, ts, stocks_per40, ast_per40, tov_per40
FROM prospect_features
ORDER BY draft_score DESC
LIMIT 25;

 * sqlite:///wnba_draft.db
Done.


RK,Name,Team,POS,draft_score,pts_per40,ts,stocks_per40,ast_per40,tov_per40
30,Sarah Strong,CONN,F,2.5767056066527725,27.985347985347985,0.68863570810499,7.472527472527473,6.593406593406593,2.93040293040293
243,MiLaysia Fulwiley,LSU,G,2.22962359013319,26.857142857142858,0.5828373015873015,8.380952380952381,6.857142857142857,4.190476190476191
2,Hannah Hidalgo,ND,G,1.9046186301234096,27.865168539325843,0.5556551353289121,6.741573033707865,6.179775280898876,3.595505617977528
36,Avery Koenen,NDSU,F,1.8563324406905992,26.57243816254417,0.6839348079161817,3.8162544169611308,3.1095406360424027,2.1201413427561837
13,Brooklyn Meyer,SDST,F,1.7061891501000708,29.61937716262976,0.6694194194194194,3.8754325259515574,3.8754325259515574,2.768166089965398
1,Audi Crooks,ISU,C,1.593417378619174,37.2027972027972,0.6966268594175571,1.8181818181818181,2.3776223776223775,3.2167832167832167
57,Taylee Chirrick,MTST,G,1.592722945225973,24.16382252559727,0.5661463664278402,5.733788395904437,4.914675767918089,3.6860068259385663
55,Timaya Lewis-Eutsey,MRSH,G,1.5426825851604788,25.942028985507246,0.5603556223390934,5.942028985507246,5.652173913043478,4.63768115942029
3,Mikayla Blakes,VAN,G,1.5354694212817774,29.939393939393938,0.5698597268364711,4.0,5.454545454545454,3.1515151515151514
230,Antoniette Emma-Nnopu,WEB,F,1.522098250111292,21.59695817490494,0.6225885654156436,4.866920152091255,4.410646387832699,4.258555133079848


In [79]:
%%sql
DROP TABLE IF EXISTS prospect_board;

CREATE TABLE prospect_board AS
WITH ranked AS (
  SELECT
    pf.*,
    PERCENT_RANK() OVER (ORDER BY draft_score) AS pr
  FROM prospect_features pf
)
SELECT
  *,
  ROUND(pr * 100.0, 1) AS draft_percentile,

  -- draft grades
  CASE
    WHEN pr >= 0.95 THEN 'A+'
    WHEN pr >= 0.90 THEN 'A'
    WHEN pr >= 0.80 THEN 'A-'
    WHEN pr >= 0.70 THEN 'B+'
    WHEN pr >= 0.60 THEN 'B'
    WHEN pr >= 0.50 THEN 'B-'
    WHEN pr >= 0.40 THEN 'C+'
    WHEN pr >= 0.30 THEN 'C'
    WHEN pr >= 0.20 THEN 'C-'
    WHEN pr >= 0.10 THEN 'D'
    ELSE 'F'
  END AS draft_grade,

  -- draft tiers
  CASE
    WHEN pr >= 0.90 THEN 'Tier 1: Franchise'
    WHEN pr >= 0.75 THEN 'Tier 2: Lottery'
    WHEN pr >= 0.50 THEN 'Tier 3: First-round'
    WHEN pr >= 0.25 THEN 'Tier 4: Second-round'
    ELSE 'Tier 5: Camp / Development'
  END AS draft_tier

FROM ranked;

 * sqlite:///wnba_draft.db
Done.
Done.


[]

In [80]:
%%sql
SELECT
  RK, Name, Team, POS,
  draft_score,
  draft_percentile,
  draft_grade,
  draft_tier,
  pts_per40, ts, stocks_per40, ast_per40, tov_per40
FROM prospect_board
ORDER BY draft_score DESC
LIMIT 30;

 * sqlite:///wnba_draft.db
Done.


RK,Name,Team,POS,draft_score,draft_percentile,draft_grade,draft_tier,pts_per40,ts,stocks_per40,ast_per40,tov_per40
30,Sarah Strong,CONN,F,2.5767056066527725,100.0,A+,Tier 1: Franchise,27.985347985347985,0.68863570810499,7.472527472527473,6.593406593406593,2.93040293040293
243,MiLaysia Fulwiley,LSU,G,2.22962359013319,100.0,A+,Tier 1: Franchise,26.857142857142858,0.5828373015873015,8.380952380952381,6.857142857142857,4.190476190476191
2,Hannah Hidalgo,ND,G,1.9046186301234096,99.9,A+,Tier 1: Franchise,27.865168539325843,0.5556551353289121,6.741573033707865,6.179775280898876,3.595505617977528
36,Avery Koenen,NDSU,F,1.8563324406905992,99.9,A+,Tier 1: Franchise,26.57243816254417,0.6839348079161817,3.8162544169611308,3.1095406360424027,2.1201413427561837
13,Brooklyn Meyer,SDST,F,1.7061891501000708,99.9,A+,Tier 1: Franchise,29.61937716262976,0.6694194194194194,3.8754325259515574,3.8754325259515574,2.768166089965398
1,Audi Crooks,ISU,C,1.593417378619174,99.8,A+,Tier 1: Franchise,37.2027972027972,0.6966268594175571,1.8181818181818181,2.3776223776223775,3.2167832167832167
57,Taylee Chirrick,MTST,G,1.592722945225973,99.8,A+,Tier 1: Franchise,24.16382252559727,0.5661463664278402,5.733788395904437,4.914675767918089,3.6860068259385663
55,Timaya Lewis-Eutsey,MRSH,G,1.5426825851604788,99.8,A+,Tier 1: Franchise,25.942028985507246,0.5603556223390934,5.942028985507246,5.652173913043478,4.63768115942029
3,Mikayla Blakes,VAN,G,1.5354694212817774,99.8,A+,Tier 1: Franchise,29.939393939393938,0.5698597268364711,4.0,5.454545454545454,3.1515151515151514
230,Antoniette Emma-Nnopu,WEB,F,1.522098250111292,99.7,A+,Tier 1: Franchise,21.59695817490494,0.6225885654156436,4.866920152091255,4.410646387832699,4.258555133079848


Now let's describe the type of players to give WNBA teams a better idea to find what they're looking for. When looking for a specific player this table can be really helpful when a team needs a specific type of player. For example, my home state team the Chicago Sky have Angel Reese. Reese is a great player who commands the low post and mid-range. A really good fit for the Chicago Sky would be a wing that can shoot to give Reese more space. I won't say a guard just yet because Hailey Van Lith had a solid rookie campaign. If I'm drafting as the Sky, I'm looking for shooters and two way wings that can help anchor down our defense. See one of the queries below to see where the Sky should look in the draft!

In [81]:
%%sql
DROP TABLE IF EXISTS prospect_board_archetypes;

CREATE TABLE prospect_board_archetypes AS
SELECT
  pb.*,

  CASE
    --------------------------------------------------
    -- ELITE SHOOTERS (priority classification)
    --------------------------------------------------
    WHEN pb.z_spacing >= 1.3 AND pb.ts >= 0.55 AND pb.three_rate >= 0.35
      THEN 'Movement Shooter'
    WHEN pb.z_spacing >= 1.0 AND pb.ts >= 0.53 AND pb.three_rate >= 0.30
      THEN 'Spot-Up Shooter'

    --------------------------------------------------
    -- Guards
    --------------------------------------------------
    WHEN pb.POS = 'G' AND pb.z_scoring >= 1.0 AND pb.z_playmaking >= 1.0
      THEN 'Playmaking Scorer'
    WHEN pb.POS = 'G' AND pb.z_scoring >= 1.0 AND pb.z_playmaking < 0.5
      THEN 'Pure Scorer'
    WHEN pb.POS = 'G' AND pb.z_playmaking >= 1.0 AND pb.z_scoring < 0.8
      THEN 'Primary Playmaker'
    WHEN pb.POS = 'G' AND pb.z_defense >= 1.0 AND pb.z_spacing >= 0.5
      THEN '3&D Guard'
    WHEN pb.POS = 'G' AND pb.z_defense >= 1.0 AND pb.z_scoring >= 0.5
      THEN 'Two-Way Guard'
    WHEN pb.POS = 'G' AND pb.z_rim_pressure >= 1.0
      THEN 'Slashing Guard'

    --------------------------------------------------
    -- Forwards
    --------------------------------------------------
    WHEN pb.POS = 'F' AND pb.z_scoring >= 1.0 AND pb.z_defense >= 0.8
      THEN 'Two-Way Wing'
    WHEN pb.POS = 'F' AND pb.z_spacing >= 1.0 AND pb.z_defense >= 0.5
      THEN '3&D Wing'
    WHEN pb.POS = 'F' AND pb.z_playmaking >= 0.9 AND pb.z_defense >= 0.6
      THEN 'Point Forward'
    WHEN pb.POS = 'F' AND pb.z_scoring >= 1.0 AND pb.z_rim_pressure >= 0.8 AND pb.three_rate < 0.25
      THEN 'Dominant Post'
    WHEN pb.POS = 'F' AND pb.z_rebounding >= 1.0 AND pb.z_defense >= 0.6
      THEN 'Rebounding Defender'

    --------------------------------------------------
    -- Centers / Bigs
    --------------------------------------------------
    WHEN pb.POS = 'C' AND pb.z_spacing >= 1.0
      THEN 'Stretch Big Shooter'
    WHEN pb.POS = 'C' AND pb.z_defense >= 1.0 AND pb.blk_per40 >= 2.5
      THEN 'Rim Protector'
    WHEN pb.POS = 'C' AND pb.z_rebounding >= 1.0 AND pb.z_scoring >= 0.5
      THEN 'Interior Force'
    WHEN pb.POS = 'C' AND pb.z_scoring >= 1.0 AND pb.z_rim_pressure >= 0.8 AND pb.three_rate < 0.20
      THEN 'Dominant Post'

    --------------------------------------------------
    -- Fallbacks 
    --------------------------------------------------
    WHEN pb.z_defense >= 1.0
      THEN 'Defensive Specialist'
    WHEN pb.z_scoring >= 1.0
      THEN 'Scoring Specialist'
    WHEN pb.z_spacing >= 1.0
      THEN 'Floor Spacer'
    ELSE 'Balanced Contributor'
  END AS archetype

FROM prospect_board pb;

 * sqlite:///wnba_draft.db
Done.
Done.


[]

In [82]:
%%sql
SELECT
  RK, Name, Team, POS,
  draft_score, draft_grade, draft_tier,
  archetype
FROM prospect_board_archetypes
ORDER BY draft_score DESC
LIMIT 30;

 * sqlite:///wnba_draft.db
Done.


RK,Name,Team,POS,draft_score,draft_grade,draft_tier,archetype
30,Sarah Strong,CONN,F,2.5767056066527725,A+,Tier 1: Franchise,Two-Way Wing
243,MiLaysia Fulwiley,LSU,G,2.22962359013319,A+,Tier 1: Franchise,Playmaking Scorer
2,Hannah Hidalgo,ND,G,1.9046186301234096,A+,Tier 1: Franchise,Playmaking Scorer
36,Avery Koenen,NDSU,F,1.8563324406905992,A+,Tier 1: Franchise,Two-Way Wing
13,Brooklyn Meyer,SDST,F,1.7061891501000708,A+,Tier 1: Franchise,Two-Way Wing
1,Audi Crooks,ISU,C,1.593417378619174,A+,Tier 1: Franchise,Interior Force
57,Taylee Chirrick,MTST,G,1.592722945225973,A+,Tier 1: Franchise,Playmaking Scorer
55,Timaya Lewis-Eutsey,MRSH,G,1.5426825851604788,A+,Tier 1: Franchise,Playmaking Scorer
3,Mikayla Blakes,VAN,G,1.5354694212817774,A+,Tier 1: Franchise,Playmaking Scorer
230,Antoniette Emma-Nnopu,WEB,F,1.522098250111292,A+,Tier 1: Franchise,Two-Way Wing


Chicago Sky needs some help surrounding Angel Reese! They NEED shooters and wings who can defend. 

In [83]:
%%sql
SELECT Name, POS, draft_score, draft_grade, archetype
from prospect_board_archetypes
where archetype in ('Two-Way Wing', 'Movement Shooter', 'Playmaking Scorer')
order by draft_score desc
limit 30


 * sqlite:///wnba_draft.db
Done.


Name,POS,draft_score,draft_grade,archetype
Sarah Strong,F,2.5767056066527725,A+,Two-Way Wing
MiLaysia Fulwiley,G,2.22962359013319,A+,Playmaking Scorer
Hannah Hidalgo,G,1.9046186301234096,A+,Playmaking Scorer
Avery Koenen,F,1.8563324406905992,A+,Two-Way Wing
Brooklyn Meyer,F,1.7061891501000708,A+,Two-Way Wing
Taylee Chirrick,G,1.592722945225973,A+,Playmaking Scorer
Timaya Lewis-Eutsey,G,1.5426825851604788,A+,Playmaking Scorer
Mikayla Blakes,G,1.5354694212817774,A+,Playmaking Scorer
Antoniette Emma-Nnopu,F,1.522098250111292,A+,Two-Way Wing
Kaety L'Amoreaux,G,1.456193979580525,A+,Playmaking Scorer


# A player to keep your eyes out for

Berry Wallace is a sophomore at the University of Illinois she is having a breakout second season. The 2024 All-American already has a draft grade of an A- as a sophomore. As she continues to grow and improve as a player, it would smart to keep an eye out for her draft stock when she is eligible for the WNBA Draft. 

In [84]:
%%sql
SELECT
    pb.Name,
    pb.POS,
    pb.draft_score,
    pb.draft_grade,
    pb.archetype,
    p.team_abbr,
    p.GP,
    p.MIN,
    p.PTS
FROM prospect_board_archetypes pb
LEFT JOIN prospects p
    ON pb.Name = p.player_name
WHERE pb.Name = 'Berry Wallace';

 * sqlite:///wnba_draft.db
Done.


Name,POS,draft_score,draft_grade,archetype,team_abbr,GP,MIN,PTS
Berry Wallace,F,0.6221464056833679,A-,Scoring Specialist,ILL,21,35.0,19.3


## Conclusion

This project demonstrates how statistical modeling can be used to translate raw box score data into a structured, interpretable draft evaluation framework. Starting from raw prospect statistics, I engineered per-40 and efficiency metrics to account for differences in playing time and usage. These metrics were then standardized using z-scores, allowing each player’s performance to be evaluated relative to the rest of the draft class across scoring, efficiency, defense, playmaking, rebounding, spacing, and ball security.

Using these standardized metrics, I constructed a composite **draft score** to rank players holistically rather than relying on any single statistic. To improve interpretability and usability, I mapped draft scores to percentiles, letter grades, and draft tiers, providing a human-readable summary of each prospect’s standing within the class. Finally, I introduced **archetype classifications** that translate statistical profiles into familiar scouting language (e.g., *Playmaking Scorer*, *Two-Way Wing*, *Dominant Post*), bridging the gap between quantitative analysis and basketball intuition.

While this model does not account for contextual factors such as team role, competition level, age, or injury history, it provides a transparent and extensible foundation for prospect evaluation. Future extensions could include position-specific standardization, usage-adjusted efficiency, lineup-based impact metrics, or clustering methods to derive archetypes automatically. Overall, this framework illustrates how data-driven methods can complement traditional scouting by offering consistent, explainable, and scalable insights into player evaluation.

This could be improved because I don't have the players ages. Prospects must be turning 22 in the calendar year of the draft. However, this project is for the love of the game. The W is in a strong spot with great early year talent, established stars, and a future of continued success!